In [1]:
### imports
import json
import import_ipynb
import ipywidgets as widgets
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ipywidgets import interact, interactive, fixed, interact_manual
from objects import *
from video import *

importing Jupyter notebook from objects.ipynb
importing Jupyter notebook from video.ipynb
importing Jupyter notebook from landmarks.ipynb
importing Jupyter notebook from plotting.ipynb


## Metadata

In [2]:
Meta = metadata('train')
MetaS = metadata('supplemental')

In [3]:
%pprint

Pretty printing has been turned OFF


## Selecting datafiles

In [4]:
file = str(Meta.files[1])
Datafile = datafile(Meta, file)

## Selecting videos

In [5]:
vid = Datafile.videos[1]
V = video(Datafile, vid)

In [6]:
print(V)

 Sequence: 1784574169
 Phrase: +44-645-14-4280-31
 Signer: 89
 Frames: 249
 Dominant Hand: right
 Percentage: 98.39


In [7]:
# Good = []
# for i,vid in enumerate(Datafile.videos):
    # V = video(Datafile, vid)
    # p = percentage(V)
    # if p == 100.00: 
        # Good += [vid]
    # print(i,p,len(Good))

In [8]:
Good = [1784587545, 1786414699, 1786455245, 1786973032, 1787094692, 1791358393, 1792308520, 
        1792909008, 1794263510, 1795294511, 1797338021, 1797425392, 1799533211, 1801891222, 
        1802098574, 1803766841, 1805308488, 1805565959, 1806516226, 1807475031, 1807505088, 
        1808538895, 1810669537, 1811016240, 1813227209, 1814409379, 1814575380, 1815284952, 1815485319]

In [9]:
V = video(Datafile, Good[1])

In [10]:
f = show_hands(V)
interact(f, frame=widgets.IntSlider(min=0, max=len(V.frames)-1, step=1, value=0, layout=widgets.Layout(width='1000px')))

interactive(children=(IntSlider(value=0, description='frame', layout=Layout(width='1000px'), max=92), Output()…

<function show_hands.<locals>.show_frame at 0x2a3eeaf80>

In [11]:
print(V)

 Sequence: 1786414699
 Phrase: 922 co 20/33
 Signer: 135
 Frames: 93
 Dominant Hand: right
 Percentage: 100.0


In [12]:
A = get_landmarks(V, 'right_hand', 0)
A = shift(A, np.array([0.,0.,0.]))

In [17]:
N,_ = normal_hand_data(V)
tf.reshape(N[0],(20,3))

<tf.Tensor: shape=(20, 3), dtype=float64, numpy=
array([[ 0.04232334, -0.05140309, -0.00675568],
       [ 0.06987115, -0.10248778, -0.01491452],
       [ 0.09462891, -0.14582631, -0.02732628],
       [ 0.11760409, -0.19613001, -0.03962396],
       [ 0.04541665, -0.14881295, -0.0078475 ],
       [ 0.05289695, -0.21441564, -0.01337965],
       [ 0.05790362, -0.2578223 , -0.02114287],
       [ 0.06898841, -0.29438388, -0.0262141 ],
       [ 0.00673874, -0.14632186, -0.01946258],
       [-0.00126522, -0.21573205, -0.02682741],
       [-0.00751232, -0.26703024, -0.03542777],
       [-0.00825011, -0.31157932, -0.03851737],
       [-0.03394313, -0.13555835, -0.03898969],
       [-0.05455344, -0.19554633, -0.05219285],
       [-0.06751906, -0.23921616, -0.06134848],
       [-0.07700428, -0.27252513, -0.06554198],
       [-0.07221762, -0.121435  , -0.06297669],
       [-0.10752161, -0.164856  , -0.08233459],
       [-0.1283358 , -0.19055426, -0.09358383],
       [-0.14583156, -0.21103041, -0.10

In [14]:
butt

NameError: name 'butt' is not defined

In [ ]:
# change label
def changelabel(self, i, letter):
    self.data_labels[i] = letter
    
#for i in range(88,92):
    #changelabel(V,i,'r')

In [ ]:
# pickle function
def pickle(D,V):
    seq_id = str(V.id)
    Frames = V.frames
    D[seq_id] = [ label(V,i) for i in Frames ]

In [ ]:
# pickle(train[str(Datafile.id)],V)

In [ ]:
#N, p = normal_hand_data(V)
# pN = V.data_labels

# loop
# for i in range(len(N)):
    # if pN[i] != '?':
        # X += [ N[i] ]
        # y += [ Encode( pN[i] ) ]

In [ ]:
# load Data
# with open('LabeledData/LabeledData' + str(Datafile.id) + '.json', 'r') as f:
with open('Labels/Labels105143404.json', 'r') as f:
    train = json.load(f)

X, y = [], []
# loop over file ids
for file in train:
    # create datafile
    Df = datafile(Meta, file)
    videos = train[file]
    for vid in videos:
        
        # load date
        V = video(Df, int(vid))
        N, p = normal_hand_data(V)
        pN = videos[vid]
        
        # loop
        for i in range(len(N)):
            if pN[i] != '?':
                X += [ N[i] ]
                y += [ Encode( pN[i] ) ]

In [ ]:
X[0][0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [ ]:
# Model
k = 59
M = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 3)),
    tf.keras.layers.Dense(6*k, activation='relu'),
    tf.keras.layers.Dense(k)])

# Compile Model
M.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
# fit model
M.fit( X, y, epochs = 400, verbose=0 )
# M.fit( X_train, y_train, epochs = 450, verbose=0 )

In [ ]:
Loss, Acctrain = M.evaluate(X, y, verbose=2)
print('accuracy:', Acctrain)

In [ ]:
Loss, Acctrain = M.evaluate(X_train, y_train, verbose=2)
Loss, Acctest = M.evaluate(X_test, y_test, verbose=2)
print('\nTrain accuracy:', Acctrain)
print('\nTest accuracy:', Acctest)

In [ ]:
# pickling weights 
M.save_weights("model.weights")

In [ ]:
print(tf.__version__)

In [ ]:
M.load_weights('model.weights')

In [ ]:
Label(V,M)

In [ ]:
with open('Labels/Labels105143404.json', 'r') as f:
    pickles = json.load(f)

In [ ]:
# pickle function
def pickle(Dict, V):
    seq_id = str(V.id)
    Frames = V.frames
    Dict[seq_id] = [ label(V,i) for i in Frames ]

In [ ]:
# pickling good movies
# pepper = {}
# for vid in Good:
    # V = video(Datafile, vid)
    # Label(V,M)
    # pickle( pepper, V )
# D = { str(Datafile.id) : pepper }

In [ ]:
with open('Labels' + str(Datafile.id) + '.json', 'w') as f:
    json.dump(train, f, indent=2)

In [ ]:
file = [i for i in train][0]
A = train[file]

In [ ]:
keys = [i for i in A]

In [ ]:
seq = keys[0]
V = video(Df, int(seq))
V.data_labels = A[seq]